In [ ]:
# In this notebook, model is trained on very lesser no of images(only 200).
# for better result ..use more images and few more epochs.

In [179]:
import os
import cv2
import keras
import random
import tensorflow
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

In [180]:
# directories of cat and dog images
direc = '/home/sourabh/Documents'
cat_dir = '/CAT/kitty'
dog_dir = '/DOG/doggy'

In [181]:
len(os.listdir(direc+cat_dir)) #cats
len(os.listdir(direc+dog_dir)) #dogs
x1=os.listdir(direc+cat_dir)
x2= os.listdir(direc+dog_dir)
X=[]
X=x1+x2
X=np.asarray(X)
X.shape

(2022,)

In [182]:
X.shape[0]

2022

In [183]:
# one_hot_encodong the Y_labels
Y=np.zeros((X.shape[0]))
for i in range(X.shape[0]):
    if X[i].startswith('cat'):
        Y[i]+=0
    if X[i].startswith('dog'):
        Y[i]+=1

In [184]:
print(Y.shape)
Y=Y.reshape((Y.shape[0],1))
_Y=np.vstack((Y[:101],Y[1010:1110]))
print(_Y.shape)

(2022,)
(201, 1)


In [186]:
cd /home/sourabh/Documents/CAT/kitty

/home/sourabh/Documents/CAT/kitty


In [187]:
_X=np.zeros((_Y.shape[0],67500))

In [189]:
# taking only 100 of cats pictures out of 1000s

for i in range(101):
    img=cv2.imread(X[i])
    img=img.reshape((67500))
    _X[i]+=img

In [190]:
cd /home/sourabh/Documents/DOG/doggy

/home/sourabh/Documents/DOG/doggy


In [191]:
# taking only 100 of dogs pictures out of 1000s

for i in range(1010,1110):
    img=cv2.imread(X[i])
    img=img.reshape((67500))
    _X[i-909]+=img

In [192]:
_X.shape

(201, 67500)

In [193]:
_X=_X/255 # rescaling the _X in range of [0,1]

In [194]:
_X

array([[0.44705882, 0.14509804, 0.04313725, ..., 0.79607843, 0.44313725,
        0.30588235],
       [0.22745098, 0.36078431, 0.56862745, ..., 0.2627451 , 0.39215686,
        0.56862745],
       [0.23921569, 0.23529412, 0.49411765, ..., 0.58039216, 0.55686275,
        0.56078431],
       ...,
       [0.57647059, 0.74117647, 0.94901961, ..., 0.30588235, 0.39607843,
        0.48235294],
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [0.12941176, 0.23137255, 0.28627451, ..., 0.25882353, 0.4627451 ,
        0.62352941]])

In [259]:
# cats :0:1009
# dogs :1010:2021

In [260]:
_X.shape, _Y.shape

((201, 150, 150, 3), (201, 1))

In [261]:
_X = _X.reshape((201,150,150,3))

In [262]:
_X.shape, _Y.shape

((201, 150, 150, 3), (201, 1))

# splitting data

In [199]:
# splitting data into test and train sets
x_train,x_test,y_train,y_test = train_test_split(_X,_Y,test_size=0.2,random_state=1,shuffle=True)

In [200]:
x_train.shape,y_train.shape,x_test.shape, y_test.shape

((160, 150, 150, 3), (160, 1), (41, 150, 150, 3), (41, 1))

# Model

In [245]:
model = Sequential()

In [246]:
c1 = Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),padding='SAME',input_shape=(150,150,3),activation='relu')
c2 = Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),padding='SAME',activation='relu')
c3 = Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),padding='SAME',activation='relu')
m1 = MaxPooling2D(pool_size=(2, 2), strides=None, padding='SAME')
m2 = MaxPooling2D(pool_size=(2, 2), strides=None, padding='SAME')
m3 = MaxPooling2D(pool_size=(2, 2), strides=None, padding='SAME')
f = Flatten()
d1 = Dense(units=64,activation='sigmoid')
d2 = Dense(units=1,activation='sigmoid')

In [247]:
# adding all the layers to the model

model.add(c1)
model.add(m1)
model.add(c2)
model.add(m2)
model.add(c3)
model.add(m3)
model.add(f)
model.add(d1)
model.add(d2)

In [248]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 150, 150, 32)      896       
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 75, 75, 32)        9248      
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 38, 38, 32)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 38, 38, 32)        9248      
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 19, 19, 32)        0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 11552)             0         
__________

In [249]:
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['acc'])

In [250]:
# Data Augmentation :-
gen = ImageDataGenerator(rotation_range=20,
                         width_shift_range=0.2,
                         height_shift_range=0.2,)

In [251]:
gen.fit(x_train)

In [252]:
model.fit_generator(gen.flow(x_train, y_train, batch_size=32),
                    steps_per_epoch=len(x_train) / 32, epochs=50)

Epoch 1/50
5/5 [==============================] - 8s 2s/step - loss: 0.7421 - acc: 0.5500
Epoch 2/50
5/5 [==============================] - 4s 869ms/step - loss: 0.7118 - acc: 0.5000
Epoch 3/50
5/5 [==============================] - 4s 872ms/step - loss: 0.7487 - acc: 0.4875
Epoch 4/50
5/5 [==============================] - 4s 867ms/step - loss: 0.7113 - acc: 0.5062
Epoch 5/50
5/5 [==============================] - 4s 875ms/step - loss: 0.6912 - acc: 0.5187
Epoch 6/50
5/5 [==============================] - 4s 864ms/step - loss: 0.7061 - acc: 0.5563
Epoch 7/50
5/5 [==============================] - 4s 881ms/step - loss: 0.7071 - acc: 0.5250
Epoch 8/50
5/5 [==============================] - 4s 877ms/step - loss: 0.6769 - acc: 0.6437
Epoch 9/50
5/5 [==============================] - 4s 869ms/step - loss: 0.7086 - acc: 0.5375
Epoch 10/50
5/5 [==============================] - 4s 855ms/step - loss: 0.6694 - acc: 0.6188
Epoch 11/50
5/5 [==============================] - 4s 861ms/step - loss:

In [265]:
outs=model.predict(x_test)
for i in range(outs.shape[0]):
    if outs[i]>=0.5:
        outs[i]=1
    else: outs[i]=0

In [272]:
(outs==y_test).sum() # no of correct predictions

19